In [2]:
import time
import numpy as np
from easyAI import TwoPlayerGame, AI_Player, Negamax, NonRecursiveNegamax, SSS, DUAL

In [3]:
class AIPlayerWithTimer(AI_Player):
    def __init__(self, AI_algo, name="AI"):
        super().__init__(AI_algo, name)
        self.move = {}
        self.mean_time = 0
        self.moves_counter = 0

    def ask_move(self, game):
        st = time.time()
        move = self.AI_algo(game)
        et = time.time()
        self.mean_time = ((self.mean_time * self.moves_counter) + (et - st)) / (self.moves_counter + 1)
        self.moves_counter += 1
        return move

In [4]:
class ClumsyConnectFour(TwoPlayerGame):
    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board if (board is not None) else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1
        self.pos_dir = np.array(
            [[[i, 0], [0, 1]] for i in range(6)]
            + [[[0, i], [1, 0]] for i in range(7)]
            + [[[i, 0], [1, 1]] for i in range(1, 3)]
            + [[[0, i], [1, 1]] for i in range(4)]
            + [[[i, 6], [1, -1]] for i in range(1, 3)]
            + [[[0, i], [1, -1]] for i in range(3, 7)]
        )

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        rand_move = np.random.choice(np.arange(-1, 2), p=[0.05, 0.9, 0.05])
        column = max(min(column + rand_move, 6), 0)
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return self.find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

    def find_four(self, board, current_player):
        for pos, direction in self.pos_dir:
            streak = 0
            while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
                if board[pos[0], pos[1]] == current_player:
                    streak += 1
                    if streak == 4:
                        return True
                else:
                    streak = 0
                pos = pos + direction
        return False

In [5]:
class ConnectFour(TwoPlayerGame):
    def __init__(self, players, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = 1
        self.pos_dir = np.array(
            [[[i, 0], [0, 1]] for i in range(6)]
            + [[[0, i], [1, 0]] for i in range(7)]
            + [[[i, 0], [1, 1]] for i in range(1, 3)]
            + [[[0, i], [1, 1]] for i in range(4)]
            + [[[i, 6], [1, -1]] for i in range(1, 3)]
            + [[[0, i], [1, -1]] for i in range(3, 7)]
        )

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )

    def lose(self):
        return self.find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

    def find_four(self, board, current_player):
        for pos, direction in self.pos_dir:
            streak = 0
            while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
                if board[pos[0], pos[1]] == current_player:
                    streak += 1
                    if streak == 4:
                        return True
                else:
                    streak = 0
                pos = pos + direction
        return False

In [6]:
class Tournament:
    def __init__(self, tournament_length, algo1, algo2, prob):
        self.ranking = [0, 0, 0]
        self.tournament_length = tournament_length
        self.player1 = AIPlayerWithTimer(algo1)
        self.player2 = AIPlayerWithTimer(algo2)
        self.prob = prob

    def start_round(self, switch_player):
        if self.prob:
            game = ClumsyConnectFour([self.player1, self.player2])
        else:
            game = ConnectFour([self.player1, self.player2])
        if switch_player:
            game.switch_player()
        game.play(verbose=False)
        if game.lose():
            self.ranking[game.opponent_index - 1] += 1
        else:
            self.ranking[-1] +=1

    def start_tournament(self):
        for _ in range(self.tournament_length):
            switch_player = np.random.choice(np.arange(0, 2), p=[0.5, 0.5])
            self.start_round(switch_player)

# Wersja probabilistyczna

### Negamax (4) vs Negamax (2)

In [10]:
tournament = Tournament(40, Negamax(4), Negamax(2), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[20, 18, 2] 0.03190418218760558 0.004641701915476583


### Negamax (3) vs Negamax (5)

In [15]:
tournament = Tournament(40, Negamax(3), Negamax(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 21, 1] 0.01440441762673877 0.09578366724999393


### DUAL (3) vs DUAL (4)

In [16]:
tournament = Tournament(40, DUAL(3), DUAL(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[12, 23, 5] 0.019328924944726874 0.04024292398476557


### DUAL (2) vs DUAL (5)

In [17]:
tournament = Tournament(40, DUAL(2), DUAL(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[22, 17, 1] 0.006034370368199222 0.2315103688539889


### Negamax (5) vs DUAL (5)

In [18]:
tournament = Tournament(40, Negamax(5), DUAL(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[27, 8, 5] 0.09137560975905278 0.10027692109718254


### Negamax (3) vs DUAL (3)

In [19]:
tournament = Tournament(40, Negamax(3), DUAL(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 15, 7] 0.013642688829204292 0.0362829383538694


### SSS (3) vs SSS (5)

In [20]:
tournament = Tournament(40, SSS(3), SSS(5), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 19, 4] 0.021972253251431595 0.13718371827210948


### SSS (2) vs SSS (4)

In [21]:
tournament = Tournament(40, SSS(2), SSS(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 20, 3] 0.06268813475122986 0.03895282297206107


### SSS (3) vs Negamax (3)

In [22]:
tournament = Tournament(40, SSS(3), Negamax(3), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[18, 17, 5] 0.01924131680046723 0.014487912098141702


### SSS (4) vs DUAL (4)

In [23]:
tournament = Tournament(40, SSS(4), DUAL(4), True)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[14, 19, 7] 0.0397767257864776 0.08151691480116409


# Wersja deterministyczna

### Negamax (4) vs Negamax (2)

In [36]:
tournament = Tournament(40, Negamax(4), Negamax(2), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[19, 21, 0] 0.023223282475220538 0.003629110047691747


### Negamax (3) vs Negamax (5)

In [ ]:
tournament = Tournament(40, Negamax(3), Negamax(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

### DUAL (3) vs DUAL (4)

In [39]:
tournament = Tournament(40, DUAL(3), DUAL(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[13, 27, 0] 0.012206828259081252 0.026740777604232022


### DUAL (2) vs DUAL (5)

In [41]:
tournament = Tournament(40, DUAL(2), DUAL(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[23, 17, 0] 0.004613579047421746 0.07065316844665863


### Negamax (5) vs DUAL (5)

In [42]:
tournament = Tournament(40, Negamax(5), DUAL(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[24, 16, 0] 0.06738046977830976 0.07103435274051584


### Negamax (3) vs DUAL (3)

In [43]:
tournament = Tournament(40, Negamax(3), DUAL(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[21, 15, 4] 0.013630356286701405 0.031074365538746224


### SSS (3) vs SSS (5)

In [44]:
tournament = Tournament(40, SSS(3), SSS(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[16, 24, 0] 0.01380847346398138 0.09787842278839437


### SSS (2) vs SSS (4)

In [48]:
tournament = Tournament(40, SSS(2), SSS(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[19, 21, 0] 0.004566761807069763 0.027423109924584105


### SSS (3) vs Negamax (3)

In [49]:
tournament = Tournament(40, SSS(3), Negamax(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[14, 26, 0] 0.013627314881274574 0.01082228077085394


### SSS (4) vs DUAL (4)

In [50]:
tournament = Tournament(40, SSS(4), DUAL(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[19, 21, 0] 0.028250686327616372 0.028047224548127917


### Negamax alfa-beta (4) vs Negamax alfa-beta (2)

In [8]:
tournament = Tournament(40, Negamax(4, win_score=3), Negamax(2, win_score=3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[22, 18, 0] 0.02238127965676159 0.0036110765055606233


### Negamax alfa-beta (3) vs SSS (4)

In [9]:
tournament = Tournament(40, Negamax(3, win_score=3), SSS(4), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[17, 23, 0] 0.010439823325196954 0.02589711943994781


### Negamax alfa-beta (3) vs DUAL (5)

In [10]:
tournament = Tournament(40, Negamax(3, win_score=3), SSS(5), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[24, 16, 0] 0.01041333182998325 0.099895845612754


### Negamax alfa-beta (3) vs Negamax (3)

In [11]:
tournament = Tournament(40, Negamax(3, win_score=3), Negamax(3), False)
tournament.start_tournament()
print(tournament.ranking, tournament.player1.mean_time, tournament.player2.mean_time)

[21, 19, 0] 0.010099453361410837 0.010394373065546952
